# Neural networks
Good article of the basics: https://victorzhou.com/blog/intro-to-neural-networks/   
Great video of the basics: https://www.youtube.com/watch?v=aircAruvnKk

### Introduction to Neural Networks.


A next step from a single perceptron -as seen in the last lecture- to an image classifier as above is a Multi-layer Perceptron.

![](http://www.saedsayad.com/images/Perceptron_bkp_1.png)



The code in the following is adapted from Chapter 18 "Neural Networks" in the Data Science from Scratch book. The code can be found at: https://github.com/joelgrus/data-science-from-scratch/blob/master/scratch/neural_networks.py


In [ ]:
%reload_ext autoreload

In [ ]:
import random
from tqdm import tqdm

random.seed(0)   # to get repeatable results
input_size = 25  # each input is a vector of length 25

num_hidden = 5   # we'll have 5 neurons in the hidden layer
output_size = 10 # we need 10 outputs for each input

# each hidden neuron has one weight per input, plus a bias weight
hidden_layer = [[random.random() for _ in range(input_size + 1)]
                 for _ in range(num_hidden)]
print(hidden_layer)

# each output neuron has one weight per hidden neuron, plus a bias weight
output_layer = [[random.random() for _ in range(num_hidden + 1)]
                 for _ in range(output_size)]
print(output_layer)

# the network starts out with random weights
network = [hidden_layer, output_layer]


print('Training...')

# 10,000 iterations seems enough to converge
for _ in tqdm(range(20000)):
    for input_vector, target_vector in zip(inputs, targets):
        backpropagate(network, input_vector, target_vector)

## A Trainings Dataset

We want to create a Multi-layer Perceptron, which can classifiy -or recognize- the digits from zero to nine for us. In `raw_digits` we create digits consisting out of 5x5 binary pixels. Consequently, each input in our trainings dataset is a binary vector of length 25.

In [ ]:
raw_digits = [
        """11111
           1...1
           1...1
           1...1
           11111""",
        """..1..
           ..1..
           ..1..
           ..1..
           ..1..""",
        """11111
           ....1
           11111
           1....
           11111""",
        """11111
           ....1
           11111
           ....1
           11111""",
        """1...1
           1...1
           11111
           ....1
           ....1""",
        """11111
           1....
           11111
           ....1
           11111""",
        """11111
           1....
           11111
           1...1
           11111""",
        """11111
           ....1
           ....1
           ....1
           ....1""",
        """11111
           1...1
           11111
           1...1
           11111""",
        """11111
           1...1
           11111
           ....1
           11111"""]


def make_digit(raw_digit):
    return [1 if c == '1' else 0
            for row in raw_digit.split("\n")
            for c in row.strip()]


inputs = [make_digit(raw_digit) for raw_digit in raw_digits]
inputs

In [ ]:
targets = [[1 if i == j else 0 for i in range(10)] for j in range(10)]
targets

For convience and reusabilty, we save the vectors containing the raw digits to a CSV file.

In [ ]:
import numpy as np


print(np.array(inputs, dtype=np.int8))
#np.savetxt?
np.savetxt('./simple_digit_trainingset.csv', np.array(inputs, dtype=np.int8), delimiter=',', fmt='%d')

In [ ]:
%%bash
cat simple_digit_trainingset.csv

We create to helper functions, one for reading our trainings dataset from a file and a second one, which will plot it for inspection.

In [ ]:
%matplotlib inline


import csv
import webget
import numpy as np
import matplotlib.pyplot as plt


filename = './simple_digit_trainingset.csv'


def read_data(filename):
    data = []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            label = reader.line_num - 1
            image = np.array(row[:], dtype=np.int8)
            data.append((label, image))
    return data


def generate_plot(data):
    count = 0
    f = plt.figure(figsize=(10, 5))
    for idx, row in enumerate(data):
        imarray = row[1].reshape((5, 5))
        plt.subplot(2, 5, idx + 1)
        plt.subplots_adjust(hspace=0.5)
        count += 1
        plt.title('Label = {}'.format(row[0]))
        plt.imshow(imarray, cmap='Greys', interpolation='None')
    return plt

## The Actual Implementation

In [ ]:
import math


def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))


def sigmoid(t):
    return 1 / (1 + np.exp(-t))


def neuron_output(weights, inputs):
    return sigmoid(dot(weights, inputs))


def feed_forward(neural_network, input_vector):
    """takes in a neural network (represented as a list of lists of lists of weights)
    and returns the output from forward-propagating the input"""

    outputs = []

    for layer in neural_network:

        input_with_bias = input_vector + [1]             # add a bias input
        output = [neuron_output(neuron, input_with_bias) # compute the output
                  for neuron in layer]                   # for this layer
        outputs.append(output)                           # and remember it

        # the input to the next layer is the output of this one
        input_vector = output

    return outputs


def predict(in_put):
    return feed_forward(network, in_put)[-1]


def backpropagate(network, input_vector, target):

    hidden_outputs, outputs = feed_forward(network, input_vector)

    # the output * (1 - output) is from the derivative of sigmoid
    output_deltas = [output * (1 - output) * (output - target[i])
                     for i, output in enumerate(outputs)]

    # adjust weights for output layer (network[-1])
    for i, output_neuron in enumerate(network[-1]):
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            # print(i,j)
            output_neuron[j] -= output_deltas[i] * hidden_output
    #print('----')
    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1 - hidden_output) *
                      dot(output_deltas, [n[i] for n in network[-1]])
                     for i, hidden_output in enumerate(hidden_outputs)]

    # adjust weights for hidden layer (network[0])
    for i, hidden_neuron in enumerate(network[0]):
        for j, in_put in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * in_put

## A Test-dataset


In [ ]:
def plot_testset(data):
    count = 0
    f = plt.figure(figsize=(10, 5))
    data = np.array(data)
    for idx, row in enumerate(data):
        imarray = row.reshape((5, 5))
        plt.subplot(2, len(data), idx + 1)
        plt.subplots_adjust(hspace=0.5)
        count += 1
        plt.imshow(imarray, cmap='Greys', interpolation='None')
    return plt


test_set = [[0,1,1,1,0,
             0,0,0,1,1,
             0,0,1,1,0,
             0,0,0,1,1,
             0,1,1,1,0],
            [0,1,1,1,0,
             1,0,0,1,1,
             0,1,1,1,0,
             1,0,0,1,1,
             0,1,1,1,0],
            [0,0,1,0,0,
             0,0,1,0,0,
             0,0,1,0,0,
             0,0,1,0,0,
             0,0,1,0,0],
            [0,1,1,0,0,
             0,0,1,0,0,
             0,0,1,0,0,
             0,0,1,0,0,
             0,0,1,0,0]]
print(test_set)
plt.show(plot_testset(test_set))

In [ ]:
for test_data in test_set:
    result = predict(test_data)
    result = np.array(result)
    print(np.argmax(result), np.array_str(result, precision=2, suppress_small=True))

In [ ]:
3 [0.   0.   0.   0.93 0.   0.   0.   0.01 0.   0.1 ]
9 [0.   0.   0.   0.   0.   0.54 0.   0.   0.91 1.  ]
1 [0.   0.96 0.03 0.02 0.   0.   0.   0.   0.   0.  ]
3 [0.   0.22 0.   0.73 0.   0.   0.   0.   0.   0.  ]

In [ ]:
for test_data in inputs:
    result = predict(test_data)
    result = np.array(result)
    print(np.argmax(result), np.array_str(result, precision=2, suppress_small=True))

# Neural Networks Done Properly...

And we still have a bit to go from classifying digits with a Multi-layer Perceptron to a Convolutional Neural Network (CNN), which is the technique that IBM applies in Watson for visual recognition. A modern framework for implementing various types of neural networks is Google's Tensorflow.


```bash
conda install theano
conda install -c conda-forge tensorflow  
conda install -c conda-forge keras
```

You can get more information about it here:

  * https://www.youtube.com/watch?v=qyvlt7kiQoI
  * https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist/#0
  * https://github.com/martin-gorner/tensorflow-mnist-tutorial/blob/master/mnist_1.0_softmax.py
  * https://en.wikipedia.org/wiki/MNIST_database




# Exercise!!!

Your task is to extend the above example to work with the 'classical' MNIST dataset, which contains many thousands of handwritten digits. Your task is to watch the video https://hooktube.com/watch?v=wQ8BIBpya2k on HookTube (a lighter version of YouTube), which gives an introduction to Google's Tensorflow - a Python framework helping to build neural networks- and you follow the tutorial on https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist.

You have to reproduce their solution and the results.

# Possible Projects:

  * Implementation of a Salient Region Detector: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.228.5552&rep=rep1&type=pdf
  * Audio Fingerprinting: http://willdrevo.com/fingerprinting-and-audio-recognition-with-python/
  * Survival on the Titanic: https://www.kaggle.com/c/titanic
  * Predict forest cover: https://www.kaggle.com/c/forest-cover-type-prediction
  * How to get free pizza: https://www.kaggle.com/c/random-acts-of-pizza